(sec:exercise-othello)=
# 演習3 - オセロエージェントを作る

今回の演習では、前節、[基本のオセロAI](sec:othello-agent)の延長として、より強いオセロのAIを作成してみよう。

前節では、オセロAIの基本であるゲーム木の探索方法について、マス評価値に基づいた[ミニマックス探索](ssec:minimax)と、マス評価値に依らない[原始モンテカルロ探索](ssec:monte-carlo)について解説した。

今回の演習では、これらにいくつかの改良を加えて、より強いオセロAIの実現を目指そう。

## 演習内容

今回の演習では以下のようなオセロプレイヤーのクラスを実装し、レベル別のオセロAIに対して7割以上の勝率を目指す。以下が実装すべき`MyPlayer`クラスのひな形である。複雑なAIを実装しない限りは、はオセロ環境が渡されてくる`MyPlayer`クラスの`play`関数のみを編集すれば十分だろう。

```python
import random

from othello import Env, Move, Player
from players.base import BasePlayer


class MyPlayer(BasePlayer):
    def __init__(self):
        pass

    def reset(self):
        """
        ゲーム開始時に行いたい処理を記述
        """

    def play(self, env: Env) -> Move:
        """
        この関数を主に更新する、以下はランダムに着手する例
        """
        moves = env.legal_moves()
        return random.choice(moves)
```

より詳細な課題作成の流れについては以下の説明に従うと入手できるテンプレート・レポジトリの[README.md](https://github.com/tatsy-classes/othello-template)に従うこと。

### 対戦相手のレベル

対戦相手の詳細な実装については公開しないが、概ね以下の方針に従った実装になっている。

- **レベル1:** 盤面の評価値を改良したミニマックス探索に基づくAI
- **レベル2:** アルファ・ベータ探索により、より多くの手を読むAI
- **レベル3:** モンテカルロ木探索において過去の計算の再利用により、探索を効率化したAI

レベル1, 2に関しては盤面の評価値をベースとしたAI、レベル3は盤面の評価値を用いず、プレイアウトとモンテカルロ木探索に基づくAIとなっている。

### ローカル環境でのテスト

本演習の採点は[GitHub Classroom](https://classroom.github.com/classrooms)を使って行う。講義中で演習用のClassroomのURLを指示するので、そのURLをブラウザで開き、テンプレート・レポジトリを自分のGitHubアカウントと紐付けること。

その後、レポジトリをcloneして、ローカル環境で`player.py`を編集してテストを実施する。テンプレート・レポジトリには、ランダムに着手するAIである`randomize.py`とミニマックス探索に基づく`minimax.py`が提供されているので、それらとテストプレイすることができる。

テストプレイには`pytest`を用いる。テンプレート・レポジトリのフォルダをターミナル環境で開き、以下のコマンドを実行する。

```shell
pytest --path "players/randomize.py"
```

`players/randomize.py`の部分を別のプレイヤーが実装されたファイル名に変更すると、そのプレイヤーに対してテストを実行される。

また`pytest`とは別に、直接、対戦環境を呼び出して、任意の2つのAIを対戦させることもできる。自分の作成したAIの強さを確かめるのに利用してほしい。

```shell
python match.py --file1 "player1.py" --file2 "player2.py" -n 100 --verbose
```

上記のコマンドで`player1.py`ならびに`player2.py`はAIを実装したPythonのスクリプトファイルに変更すること。また`-n 100`のように数字をしていすると、100回テスト対戦が行われる。最後の`--verbose`フラグは、対戦中の付加情報を表示するためのフラグである。

### 本番環境でのテスト

Github Classroomでは、演習用のレポジトリにコードをプッシュする度にテストが実行される。本番環境では、

本番環境では、各レベルのAIと合計20回対戦する。この結果、勝率が7割を超えていれば、そのレベルはクリアとなる。各レベルをクリアすると**レベル×10点**が与えられる。

この際、20回の対戦にレベル1なら1分、レベル2なら2分、レベル3なら3分以上経過すると、タイムアウトになるので、なるべく早く手を考えるAIを作成することに留意してほしい。

:::{warning}
ただし、**実行に用いられる乱数は毎回変化する**仕様にしているので、各レベルのAIに確実に7割以上の勝率を上げられるものを作らないと点数が下がる可能性があることに注意。
:::

## 課題の進め方

### ゲームAIの概観

前節、[基本のオセロAI](sec:othello-agent)では

- 人間の事前知識に基づいたセルの評価に基づく手法 (ミニマックス探索など)
- 人間の事前知識に依らない手法 (モンテカルロ木探索など)

の2タイプのAIについて紹介した。

オセロゲームの場合、任意の盤状態において着手できる合法手は10前後であるため、手の探索が十分に高速なAIであれば、アルファ・ベータ探索のような枝刈り全探索に基づく手法により、かなり強いAIを作ることができる。また、アルファ・ベータ探索に用いる盤の評価値についても、前節で紹介したセルの評価値に加えて、相手が着手できる合法手の数 (少ない方が良い)や、確定石の数を含めることが考えられる。

モンテカルロ木探索は、オセロよりも着手できる合法手が多いゲーム (将棋や囲碁など)で、そもそも全ての手を枝刈りしたとしても探索することが難しい場合に有効な手法である。また、近年、囲碁AIとして注目を集めたAlphaGoや、その発展形であるAlphaZeroは、モンテカルロ木探索と深層学習による着手の選択、盤面の評価を組み合わせた手法である。

ただし、繰り返しになるがオセロゲームは合法手の数が相対的に少ないため、着手の選択までを深層学習でやらせるべきかは悩ましいところである。実際、将棋などのゲームでは深層学習ベースの手法は形勢の判断力には優れるものの、特に最終版での手の選択にはやや劣るという側面もある。

これら2つのタイプのAIの側面を考慮した上で、どのようなAIを実装するのかを検討してみてほしい。

### 課題の始めに試すこと

まずは、初期状態で与えられている「ランダムに着手するAI」で各レベル、どのくらいの勝率が得られるかをチェックしておこう。

その上で、テンプレート・レポジトリにはミニマックス探索に基づくAIが与えられているので、これを改良して、より強いAIを作っていくのが良いだろう。

また、前節でモンテカルロ木探索に基づく着手の手法は紹介済みであるので、このコードを`MyPlayer`に移植するだけでも、それなりの強さのAIを作ることができるだろう。

### より強いAIを作るために

今回の演習では20回のテストプレイにかかる時間が最大でも3分以内とかなり短い。GitHub Actionsの実行環境では、それほど早いCPUを使うことはできないので、単にミニマックス探索の探索の深さを増やす、などの方法では簡単にタイムアウトしてしまう。

よって、ミニマックス系かモンテカルロ系のどちらについても、**効率的により良い手を見つけるにはどうすれば良いか**を考えなければならない。

#### ミニマックス系

**ミニマックス系**に関しては、まず、**アルファ・ベータ探索による枝刈り**を導入すべきだろう。おそらく、同じ計算時間で1手は多く読むことができるはずである (盤面の候補は指数関数的に増えるので、これでもかなり効率化されている)。また、アルファ・ベータ探索による枝刈りよりも効率的 (だが複雑)な枝刈りの手法として[NegaScout法](https://en.wikipedia.org/wiki/Principal_variation_search)などもあるので、興味のある読者は実装に挑戦してみてほしい。

また、アルファ・ベータ探索は、枝刈りにより処理を効率化する(つまり、それ以上評価を進めても、最大評価とならない盤面の探索をしない)手法であるため、最初の方の探索で高い評価値が得られる手順が見つかれば、相当数の盤面を調べなくて済む。この考えに基づけば、ミニマックス探索の実装になるように合法手を得られた順に調べるのは効率的とは言えない。この探索順序を工夫することで、より効率的な手の探索が行えるはずだ。

加えて、ミニマックス系は読みの深さが深くなればなるほど強くなる一方で、多くの盤面を評価しようとすると、計算上が指数関数的に大きくなるという問題がある。だが、オセロゲームは終局の直前になると、そもそも石を置ける場所が少なくなるため、仮に全探索をしたとしても、検討すべき盤面の数が飛躍的に減っていく。従って、終盤、空マスの数が少なくなったら、読みの深さを深くする、などの方法が考えられるだろう。

最後に、より良い手を見つけるために、セルの評価値に基づく盤面の評価を改善することが考えられる。オセロには、相手が着手できなければパスになるというルールがあり、また確定石と呼ばれる確実に自分の色の石になるマスが決まるという特性がある。これらの特性を考慮すると、より相手にとっての合法手が少なく、また確定石の数がより多くなるような手を優先的に調べるのが良さそうに思える。ただし、確定石の数を調べるには、それなりに計算コストがかかるため、角やエッジに探索範囲を絞るなどの工夫が必要になるだろう。

#### モンテカルロ系

**モンテカルロ系**に関して、前節のモンテカルロ木探索に基づく手法には1つ大きく改善すべき点がある。

前節の実装では、新しい手を考える時に、その都度新しくUCTのノードを作り、その子ノードに対する評価していた。しかし、多くの場合、モンテカルロ木探索で着手した結果は、それほど悪い手ではなく、探索中には相手の手も考慮しているため、相手が探索中に考えた手を指してくる可能性が高い。その意味で、過去の探索結果を捨ててしまうような上記の実装は効率的とは言えない。

そこで、過去の探索において探索して2手先の盤に対応するノードを着手時にキャッシュしておき、次に手番が回ってきたときに、キャッシュしたノードの中に対応するノードが見つかれば、そこから探索を再開する、という方法が考えられる。これにより、仮に1回の着手時に行うプレイアウトの回数が100回などに制限されていたとしても、実際にはより多くの回数プレイアウトを行ったのと同等の評価結果を得ることができる。

ただし、裏を返せば (オセロではそれほど効果的ではないが)、モンテカルロ系のAIは予想外の手を指されると、一から盤の評価をやり直さなければならなくなるため、あえてその弱点を突くような実装にするという工夫をいれることもできるかもしれない。

また、オセロは60手で確実にゲームが終わるためプレイアウトの効率は比較的良いものの、終局までプレイアウトをしないと盤面の評価値を更新できないことはマイナスとも言える。そこでプレイアウトを終局まで行う代わりに、その局面がどのくらい勝ちやすい局面なのかを機械学習する方法が考えられる。これは盤面の状態を入力とし、スカラの評価値を返すような機械学習モデルを作成すれば良いため、少しの頑張りで導入できるだろう。

最後に、上記の盤面の勝ちやすさの判定に加えて、どの手に着手するかも機械学習によって決定する、ということもできる。これはAlphaZeroが使用している方法で、モンテカルロ木探索と深層学習器による盤面の評価、着手の決定を組み合わせてプレイアウトを区入り返し、より良い評価関数と着手方策を強化学習的に得る手法である。このような考え方はミニマックス系にも導入可能ではあるが、学習の時点では純粋にゲームの勝ち負けだけを考慮するモンテカルロ系の手法を用いた方がバイアスが少ないAIが得られるだろう (が、それがイコール強いかどうかは別問題)。